# concertina button finder

## MUPHIN coding music - 2015, 2018
#### find the best path for playing any tune on a concertina

Well… here is a tune. Get your concertina. And play it. But how? Where? What keys? It is quite hard to start playing the concertina. Especially considering that quite a few notes are on several positions.

This is a tune in abc format:
<PRE>X: 1
T: Mrs Saggs
R: reel
M: 4/4
L: 1/8
K: Gmaj
| : DGA | B4 A2 GF | G6 Bd | e/2f/2gdg cgdg | g2 fe fdef |
g2 d2 cBAG | D2 C2 B,2 D2 | ECEG BAGF | G4 z  : |
| : GAB | c4 B2 AG | DFAd fdef | g2 d2 cBAG | B2 AG A2 AB |
c4 B2 AG | FA df a2 gf | g2 d2 cBAG | G4 z  : |
</PRE>


## Links
* Get your favorite tune in an abc file anywhere, for example from [thesession.org](https://thesession.org/)
* What is a concertina? Try this video of [Edel Fox at youtube](https://www.youtube.com/watch?v=axi6TrtNd_4)





In [186]:
#####################
# TODO, KNOWN BUGS  #
#####################
#  1 solved
#  2 solved
#  3 visualisation of the results   
#  4 convert all into ios/android app
#  5 solved
#  6 solved
#  7 smarter cleanup
#  8 tree instead of array with too much data
#  9 To sharpen a note precede it with the circumflex or caret ^ :: ^c
# 10 To flatten a note precede it with an underscore _ :: _B
# 11 Double sharps are shown as ^^ and double flats as __ ::
# 12 To naturalise (?) a note precede it with an equals sign = :: =c
# 13 insert while reading .abc something like  unicode_normalize(:nfkd).chars



In [187]:
# Let's start with some constant values. 
# A route is a one of the many ways to play the tune.
# $Maxroutes is the maximum number of routes this script holds in memory
# while searching for the best route. The higher, the longer calculation time.
# in general, there is no difference in results for lower values.
# tested range is 1024 ~ 16384
# Default: 4096
$Maxroutes = 4096

# true = do cleanup to the maximum defined above.
# false = no cleanup. Thats for testing only.
$cleanup=  true

# If a tune has the same note twice like A3 G3 G3 A3
# if true: script list A3 G3 A3, false: A3 G3 G3 A3
# true = better performance, less memory needed
$dupchek=  true


true

# Reading ABC
## Some basic Ruby code to get the ABC file into a hash.


In [188]:
tune={}

# READ FILE
# File.open("kesh.abc", "rb") do |f|
# File.open("mrs-saggs.abc", "rb") do |f|
 File.open("patsygearys.abc", "rb") do |f|
# File.open("joebanes-1.abc", "rb") do |f|  
# File.open("drowsymaggie-1.abc", "rb") do |f|  
#  File.open("edwardtheseventh-1.abc", "rb") do |f|  
  f.each_line do |line|
    puts line.chomp
    
    line = line.chomp  # remove \n and/or \r
    if line[1]==':' && line[0]!= '|'
      line.delete!(" ")
      infotype=line[0]      
      line=line[2..-1]
      tune[:number] = line if infotype=='X'
      tune[:title]  = line if infotype=='T'
      tune[:rhythm] = line if infotype=='R'
      tune[:meas]   = line if infotype=='M'
      tune[:notelen]= line if infotype=='L'
      tune[:key]    = line if infotype=='K'
      
    else 
      # this must be a line of notes  
      if tune.include?(:line)
        tune[:line] << line
      else
        tune[:line] = line
      end
      
    end # if/else
  end   # each
end     # file




X: 1
T: Patsy Geary's
R: slide
M: 12/8
L: 1/8
K: Dmaj
AG|:F2 A AFA|B=cB A2 G|F2 A d2 e|~f3 fef|
|~g3 fgf|efe d2 B|1ABA AFD|~E3 EAG:|2 ABA efe|~d3 d2 e||
|:f2 e f2 e|f2 e fga|ABA BAF|ABA ABd|efe efe|
|efe dfa|baf afe|1 ~d3 d2 e:|2 ~d3 d2 A|| 


#<File:patsygearys.abc (closed)>

# Theory and Algoritm
## Storing the Paths
Well, I am very sure there must be smarter ways (like trees) to store the paths, but I like to work with plain arrays and hashes.

Let's say we want to play A->G on the third octave,
If the A3 (note A on third octave) is on button 1, 2 and 3,
and G3 is on button 4 and 5:
there are 6 possible paths to play A3->G3:

<pre>[1,4] [1,5] [2,4] [2,5] [3,4] [3,5]</pre>

where [1,4] is playing first button 1, second button 4. You might already notice that this will grow quickly if searching a complete tune.

## Distance Matrix
If [1,4] is an easy step, and [1,5] is not, we prefer the [1,4].  We will need kind of a score to compare both options, and therefore we calculate a so called Distance Matrix.  It describes the level of difficulty to play note on button y after note on button x.

The difficulty of playing note y after note x depends on:

* stay on the same side (left or right): same side is a tiny little bit easier,
* stay on the same row: (c, g or top row): I don’t care, no score
* samefinger, same side = hopping: severe penalty
* prefer g and c row: g and c row preferred over top row
* prefer finger: playing on the inside button is good. Therefore a benefit if it’s the first (or second) button on a row
* switch from push to pull or vice versa:  a small penalty
* same note? than play it on the same button.

# Sharps

The ABC notation says F but (for example) it should be F# when playing in G Maj. This code adds the #'s. 

* The key of the tune is in tune[:key]

* sharps['F'] has all the keys where F should be F#, so sharpes note by checking 
<pre>
    sharps['F'].include? tune[:key]
</pre>



### Now update the tuneline with sharp-symbol# added 

In [189]:
sharps = {
  'F' => [   #~   F will be F# by default WHEN PLAYING IN...: 
    "G", "GMaj", "Gmaj", "GM",
    "D", "DMaj", "Dmaj", "DM",
    "A", "AMaj", "Amaj", "AM",
    "Em","Emin",
    "Bm","Bmin",
    "Cm","Cmin"
    ],
  'C' => [  #~   C will be C# by default WHEN PLAYING IN...:
    "D", "DMaj", "Dmaj", "DM",
    "A", "AMaj", "Amaj", "AM",
    "E", "EMaj", "Emaj", "EM",
    "Bm","Bmin"
    ],
  'G' => [  #~   G will be G# by default WHEN PLAYING IN...:
    "E", "EMaj", "Emaj", "EM",
    "A", "AMaj", "Amaj", "AM",
    ],
   }

{"F"=>["G", "GMaj", "Gmaj", "GM", "D", "DMaj", "Dmaj", "DM", "A", "AMaj", "Amaj", "AM", "Em", "Emin", "Bm", "Bmin", "Cm", "Cmin"], "C"=>["D", "DMaj", "Dmaj", "DM", "A", "AMaj", "Amaj", "AM", "E", "EMaj", "Emaj", "EM", "Bm", "Bmin"], "G"=>["E", "EMaj", "Emaj", "EM", "A", "AMaj", "Amaj", "AM"]}

# Parsing the Note-line into notes

In tune[:line] are the notes of the tune.  

Now delete all characters that are not notes:
* no numbers 
* no |:  :| 
* no | 
* no ^ -> make sharp
* no _ -> make flat
* other chars like / of [

In [190]:
# these strings/chars will be deleted without further notice:
deleteme = ["|:",":|",":","|]","[|", "0","1","2","3","4","6","8", "::", "|", "/", "~" , "=" ]

deleteme.each { |del|
  tune[:line].delete! del
  } 
puts tune[:line].delete! " "
listofnotes = tune[:line]

AGFAAFABcBAGFAdeffefgfgfefedBABAAFDEEAGABAefeddefefefefgaABABAFABAABdefeefeefedfabafafeddeddA


"AGFAAFABcBAGFAdeffefgfgfefedBABAAFDEEAGABAefeddefefefefgaABABAFABAABdefeefeefedfabafafeddeddA"

## now tune[:line] should contain all notes, and nothing else. 
### Any other character left will give an error while parsing, but it will not crash the script. 
### Now one step left before parsing: 
### prepare the conversion table ABC notation to  note/octave notation of this script (Ac\f\ -> A3,C4,F4)

In [191]:
# CONVERT ABC NOTE LIKE c' to C4
# TODO Key to Sharps.Flats
abc2parse = { 
"C,"=> "C1",
"D,"=> "D1",
"E,"=> "E1",
"F,"=> "F1",
"G,"=> "G1",
"A,"=> "A1",
"B,"=> "B1",
"C"=> "C2",
"D"=> "D2",
"E"=> "E2",
"F"=> "F2",
"G"=> "G2",
"A"=> "A2",
"B"=> "B2",
"c"=> "C3",
"d"=> "D3",
"e"=> "E3",
"f"=> "F3",
"g"=> "G3",
"a"=> "A3",
"b"=> "B3",
"c\'"=> "C4",
"d\'"=> "D4",
"e\'"=> "E4",
"f\'"=> "F4",
"g\'"=> "G4",
"a\'"=> "A4",
"b\'"=> "B4"
} 


# Now add the sharps 
# later version will also add any flats

abc2js={}
abc2parse.each {|key,value|
  
  note = key[0]
  if sharps[note.upcase] != nil   
    if sharps[note.upcase].include? tune[:key]
       abc2js[key] = value[0]+"#"+value[1]    # or .insert(1,"#")
    else 
       abc2js[key] = value
    end
  else 
    abc2js[key] = value
  end
  }

# thanks for keeping the results
abc2parse = abc2js.dup

{"C,"=>"C#1", "D,"=>"D1", "E,"=>"E1", "F,"=>"F#1", "G,"=>"G1", "A,"=>"A1", "B,"=>"B1", "C"=>"C#2", "D"=>"D2", "E"=>"E2", "F"=>"F#2", "G"=>"G2", "A"=>"A2", "B"=>"B2", "c"=>"C#3", "d"=>"D3", "e"=>"E3", "f"=>"F#3", "g"=>"G3", "a"=>"A3", "b"=>"B3", "c'"=>"C#4", "d'"=>"D4", "e'"=>"E4", "f'"=>"F#4", "g'"=>"G4", "a'"=>"A4", "b'"=>"B4"}

## Now lets make an array of the notes to play, notes will be in the notation of this script, like [A3 B3 C4]. 


#### this is very basic parsing

Start at the very beginning of  listofnotes
* try 3 chars, does it exist in the list of notes? do it!
* try 2 chars, does it exist? do it!
* try 1 char , does it exist? do it!
* Otherwise: error. Delete on char and continue




In [192]:
tune

{:number=>"1", :title=>"PatsyGeary's", :rhythm=>"slide", :meas=>"12/8", :notelen=>"1/8", :key=>"Dmaj", :line=>"AGFAAFABcBAGFAdeffefgfgfefedBABAAFDEEAGABAefeddefefefefgaABABAFABAABdefeefeefedfabafafeddeddA"}

In [193]:
a =[]

while listofnotes != nil do
  found = false
  for i in [2,1,0] do  #it should finish when found
    if not found
      if abc2parse.include? listofnotes[0..i]
        found = true 
        foundnote = listofnotes[0..i]
        
        a << abc2parse[foundnote]
        listofnotes = listofnotes[i+1..-1]    # strip i chars, keep the rest ; -1=last char of string    
      end # if include?
    end   # if notfound
  end     # for
  
  if not found    #still not found?
    puts "Thats an error on "+listofnotes[0..2]
    listofnotes = listofnotes[1..-1]
  end # if
end   # while

puts a 


["A2", "G2", "F#2", "A2", "A2", "F#2", "A2", "B2", "C#3", "B2", "A2", "G2", "F#2", "A2", "D3", "E3", "F#3", "F#3", "E3", "F#3", "G3", "F#3", "G3", "F#3", "E3", "F#3", "E3", "D3", "B2", "A2", "B2", "A2", "A2", "F#2", "D2", "E2", "E2", "A2", "G2", "A2", "B2", "A2", "E3", "F#3", "E3", "D3", "D3", "E3", "F#3", "E3", "F#3", "E3", "F#3", "E3", "F#3", "G3", "A3", "A2", "B2", "A2", "B2", "A2", "F#2", "A2", "B2", "A2", "A2", "B2", "D3", "E3", "F#3", "E3", "E3", "F#3", "E3", "E3", "F#3", "E3", "D3", "F#3", "A3", "B3", "A3", "F#3", "A3", "F#3", "E3", "D3", "D3", "E3", "D3", "D3", "A2"]


## Array a is an array of notes. 

## At this point two seperate scripts are glued together. We'll do the Easy way now: 

In [194]:
Saved_old_tune= tune
tune = a

<main>:1: warning: already initialized constant Saved_old_tune
<main>:1: warning: previous definition of Saved_old_tune was here


["A2", "G2", "F#2", "A2", "A2", "F#2", "A2", "B2", "C#3", "B2", "A2", "G2", "F#2", "A2", "D3", "E3", "F#3", "F#3", "E3", "F#3", "G3", "F#3", "G3", "F#3", "E3", "F#3", "E3", "D3", "B2", "A2", "B2", "A2", "A2", "F#2", "D2", "E2", "E2", "A2", "G2", "A2", "B2", "A2", "E3", "F#3", "E3", "D3", "D3", "E3", "F#3", "E3", "F#3", "E3", "F#3", "E3", "F#3", "G3", "A3", "A2", "B2", "A2", "B2", "A2", "F#2", "A2", "B2", "A2", "A2", "B2", "D3", "E3", "F#3", "E3", "E3", "F#3", "E3", "E3", "F#3", "E3", "D3", "F#3", "A3", "B3", "A3", "F#3", "A3", "F#3", "E3", "D3", "D3", "E3", "D3", "D3", "A2"]

# let's make a few lists

$tinakeys: the 30 buttons make 60 notes! These are mine, it can be different on yours.

In [195]:
#
# these are the concertina keys in a hash
# made in: map3.xlsx, dropbox folder 60
#

$tinakeys={
 1=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "E1"},
 2=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "A1"},
 3=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "C#2"},
 4=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "A2"},
 5=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "G#2"},
 6=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "C1"},
 7=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "G1"},
 8=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "C2"},
 9=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "E2"},
10=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "G2"},
11=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "B1"},
12=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "D2"},
13=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "G2"},
14=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "B2"},
15=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "D3"},
16=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "F1"},
17=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "BB1"},
18=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "D#2"},
19=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "G2"},
20=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "BB2"},
21=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "G1"},
22=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "B1"},
23=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "D2"},
24=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "F2"},
25=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "A2"},
26=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "A1"},
27=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "F#2"},
28=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "A2"},
29=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "C3"},
30=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "E3"},
31=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "C#3"},
32=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "C#3"},
33=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "G#3"},
34=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "C#4"},
35=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "A4"},
36=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "C3"},
37=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "E3"},
38=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "G3"},
39=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "C4"},
40=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "E4"},
41=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "G3"},
42=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "B3"},
43=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "D4"},
44=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "G4"},
45=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "D#4"},
46=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "C#3"},
47=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "D#3"},
48=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "G3"},
49=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "BB3"},
50=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "D4"},
51=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "B2"},
52=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "D3"},
53=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "F3"},
54=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "A3"},
55=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "B3"},
56=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "F#3"},
57=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "A3"},
58=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "C4"},
59=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "E4"},
60=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "F#4"}
};



{1=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>5, "pushpull"=>1, "ppname"=>"push", "note"=>"E1"}, 2=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>4, "pushpull"=>1, "ppname"=>"push", "note"=>"A1"}, 3=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>3, "pushpull"=>1, "ppname"=>"push", "note"=>"C#2"}, 4=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>2, "pushpull"=>1, "ppname"=>"push", "note"=>"A2"}, 5=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>1, "pushpull"=>1, "ppname"=>"push", "note"=>"G#2"}, 6=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>5, "pushpull"=>1, "ppname"=>"push", "note"=>"C1"}, 7=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>4, "pushpull"=>1, "ppname"=>"push", "note"=>"G1"}, 8=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>3, "pushpull"=>1, "ppname"=>"push", "note"=>"C2"}, 9=>{"side"=>1, "siden

# The numbers of the buttons
$tinakeys[15] is now:: 
{
"side"=>1, 
"sidename"=>"left", 
"row"=>3, 
"rowname"=>"g", 
"button"=>1, 
"pushpull"=>1, 
"ppname"=>"push", 
"note"=>"D3"
}


These numbers are arbitrary:
 number 1 is Left Side at top row, outer button, push.
 1..15 are all buttons on the left site, starting top row, outer button
 16..30 are the same button, but pulled.
 
 31..45 and 46..60 are the buttons on the right side, 
 starting with push again.
 

So $tinakeys[15] is the D3 
* on the left side (side=2, sidename=left)
* on the bottom row (rowname g, row 3 in this script), 
* inner button which is for finger 1, 
* on push (which is also pushpull=2 in this script). 


## $tinamap = onscreen names of buttons

In [196]:
$tinamap= Hash.new

$tinakeys.each {|h,k|
  $tinamap[h]=  k["sidename"][0].upcase+ k["rowname"][0].upcase+ k["button"].to_s+"-"+ k["ppname"].upcase+"="+ k["note"].upcase
    
  #puts $tinamap[h]
  }




{1=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>5, "pushpull"=>1, "ppname"=>"push", "note"=>"E1"}, 2=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>4, "pushpull"=>1, "ppname"=>"push", "note"=>"A1"}, 3=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>3, "pushpull"=>1, "ppname"=>"push", "note"=>"C#2"}, 4=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>2, "pushpull"=>1, "ppname"=>"push", "note"=>"A2"}, 5=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>1, "pushpull"=>1, "ppname"=>"push", "note"=>"G#2"}, 6=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>5, "pushpull"=>1, "ppname"=>"push", "note"=>"C1"}, 7=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>4, "pushpull"=>1, "ppname"=>"push", "note"=>"G1"}, 8=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>3, "pushpull"=>1, "ppname"=>"push", "note"=>"C2"}, 9=>{"side"=>1, "siden

# Now calculate distance map from button to button

## Distance from button to button: 
### the lower the value in distance, the easier the play from button -> button

### We calculate the score by comparing the two buttons.
* whether they are on the same side of the tina --> sameside(f,t)
* DELETED:: --whether they are on the same row --> samerow(f,t) ::DELETED
* whether you switch from Push to Pull --> ppchng(f,t)

### or by the attributes of the next button: 
* which row (top row, c row, g row) --> prefrow(f,t)
* preferred finger is 1, or 2. This will play on the inner buttons if possible --> preffing(f,t)

### hopping
* samefinger, same side, no switch from push to pull => hopping. Severe penalty --> hopp(f,t)



In [197]:
def sameside(f,t)
  score = 0
  if $tinakeys[f]["side"]==$tinakeys[t]["side"] 
    score = -1
  end
  result = score
end

def samerow(f,t)   # no in use anymore
  score = 0
  if $tinakeys[f]["row"]==$tinakeys[t]["row"] 
    score = 0
  end
  result = score
end

def hopp(f,t)
  score = 0
  
  # hopping defines as:
  # same finger has to jump from one button to another
  if (($tinakeys[f]["button"] == $tinakeys[t]["button"]) && 
      ($tinakeys[f]["side"] == $tinakeys[t]["side"])   )
    score = 100
  end
    
  #but... just same button push-pull, that's good, that's no hopping!
  if (($tinakeys[f]["row"]     == $tinakeys[t]["row"]) && 
      ($tinakeys[f]["side"]    == $tinakeys[t]["side"]) &&
      ($tinakeys[f]["button"]  == $tinakeys[t]["button"]) &&    
      ($tinakeys[f]["pushpull"]!= $tinakeys[t]["pushpull"]) )           
      
     score = -12  #-10
  end
  
  result = score
end

def prefrow(f,t)  
  # score = 15   # was +7, later +11. Penalty for the non c/g row
  if [2,3].include? $tinakeys[t]["row"]  # prefer c or g row
    score = 0
  else
    score = 15
  end
  result = score
end

def prefbuttt(t)
  score = 0
  if ($tinakeys[t]["button"]==1)  # prefer button one
    score = -7   #-4
  elsif
    ($tinakeys[t]["button"]==2)  # or prefer button two
    score = -2
  elsif
    ($tinakeys[t]["button"]==5)  # no button 5 please
    score = 5
  end  
  
  result = score
end

def ppchng(f,t)
  score = 1   #2
  if $tinakeys[f]["pushpull"]==$tinakeys[t]["pushpull"] #push after push = good
    score = 0
  end
  result = score
end



:ppchng

In [198]:
# create a distance matrix, init with zeros ######################################
$distmat = Array.new(61){Array.new(61,0)}

#now loop over all 60 buttons
for f in 1..60 
  for t in 1..60

      if (f == t)    # same note? perfect, same button!! 
        score = -25  
      elsif (hopp(f,t) == 100)  # hop? no!
        score = 100
      else
        score = sameside(f,t)+
                #samerow(f,t)+
                hopp(f,t)    +
                prefrow(f,t) +
                prefbuttt(t) +
                ppchng(f,t) 
      end
      $distmat[f][t] = score
    
  end
end


1..60

# now the note -> buttons list
## for example: note B3 is on two positions: 42 and 55
## Make a list of this: note "B3" is at [42,55]. 

In [199]:
nlist=[]
for i in 1..60 do
  nlist << $tinakeys[i]["note"]
end
nlist= nlist.uniq.sort


#init to make it available outside the loop
$note_keys = Hash.new( [1] ) 

nlist.each { |nownote|
  $note_keys[nownote]=[]
  }

for i in 1..60 do
  $note_keys[$tinakeys[i]["note"]] << i
end
  
$note_keys

{"A1"=>[2, 26], "A2"=>[4, 25, 28], "A3"=>[54, 57], "A4"=>[35], "B1"=>[11, 22], "B2"=>[14, 51], "B3"=>[42, 55], "BB1"=>[17], "BB2"=>[20], "BB3"=>[49], "C#2"=>[3], "C#3"=>[31, 32, 46], "C#4"=>[34], "C1"=>[6], "C2"=>[8], "C3"=>[29, 36], "C4"=>[39, 58], "D#2"=>[18], "D#3"=>[47], "D#4"=>[45], "D2"=>[12, 23], "D3"=>[15, 52], "D4"=>[43, 50], "E1"=>[1], "E2"=>[9], "E3"=>[30, 37], "E4"=>[40, 59], "F#2"=>[27], "F#3"=>[56], "F#4"=>[60], "F1"=>[16], "F2"=>[24], "F3"=>[53], "G#2"=>[5], "G#3"=>[33], "G1"=>[7, 21], "G2"=>[10, 13, 19], "G3"=>[38, 41, 48], "G4"=>[44]}

# now it's time to do a serious amount of work

### calculate score of all combinations 
### if the list of possible routes is getting too long, clean it up 

In [200]:
# a = for example [[32, 54, 57], [42, 55], [39, 58], [56]]
# a is in this case a four note-tune 
a=[]
tune.each { |note|
  a << $note_keys[note]
}
 
pr=1
a.cycle(1) { |ar| pr = pr*ar.length
  }
puts "There are "+pr.to_s+" combinations for this tune."

if $dupchek  
  # duplicate check: play same note twice?  Skip one! 
  a_new=[]
  for i in 0..a.length-2
    a_new << a[i] unless a[i] == a[i+1]
  end

  a = a_new
  pr = 1
  a.cycle(1) { |ar| pr = pr*ar.length
    }
  puts "Cleaned up! Now there are "+pr.to_s+" combinations."
end


There are 22358488275953674912530432 combinations for this tune.
Cleaned up! Now there are 8625959983006819024896 combinations.


In [201]:
a

[[4, 25, 28], [10, 13, 19], [27], [4, 25, 28], [27], [4, 25, 28], [14, 51], [31, 32, 46], [14, 51], [4, 25, 28], [10, 13, 19], [27], [4, 25, 28], [15, 52], [30, 37], [56], [30, 37], [56], [38, 41, 48], [56], [38, 41, 48], [56], [30, 37], [56], [30, 37], [15, 52], [14, 51], [4, 25, 28], [14, 51], [4, 25, 28], [27], [12, 23], [9], [4, 25, 28], [10, 13, 19], [4, 25, 28], [14, 51], [4, 25, 28], [30, 37], [56], [30, 37], [15, 52], [30, 37], [56], [30, 37], [56], [30, 37], [56], [30, 37], [56], [38, 41, 48], [54, 57], [4, 25, 28], [14, 51], [4, 25, 28], [14, 51], [4, 25, 28], [27], [4, 25, 28], [14, 51], [4, 25, 28], [14, 51], [15, 52], [30, 37], [56], [30, 37], [56], [30, 37], [56], [30, 37], [15, 52], [56], [54, 57], [42, 55], [54, 57], [56], [54, 57], [56], [30, 37], [15, 52], [30, 37], [15, 52]]

In [202]:
  # This Function calculates the score of route (array of button numbers) 
  def calcscore(arr)   
        score = startbutton(arr[0].to_i)   # the startbutton
        for i in 0..(arr.length-2)
          # fro (i)   is the note you come from,
          # too (i+1) is the note to go to
          score = score+$distmat[arr[i].to_i][arr[i+1].to_i]
        end
        result = score.to_i
    end #~ def    

:calcscore

In [203]:
    # the first note of the tune has its own score
    def startbutton(f)
      score = 0
      if ($tinakeys[f]["button"] == 1)  # prefer button one
        score = -5
      elsif
         ($tinakeys[f]["button"] == 2)  # prefer button two as well
         score = -4
      elsif
         ($tinakeys[f]["button"] == 4)  # no button 4 please
         score = 3
      elsif
         ($tinakeys[f]["button"] == 5)  # no button 5 please
         score = 5
      end  

      if [2,3].include? $tinakeys[f]["row"]   # prefer c or g row
        score = score-5
      end
      result = score
    end

    #######################################################
    # VARIABLES with influence on solving the puzzle
    #  
    # if this script doesn't find a proper solution, 
    # raise this value to 120. Default is 30.
    uppervalscore = 120
    #
    # if there's no solution, larger the sample
    # default 2.0
    $divv = 2.0
    #######################################################


# first note in tune needs initialization:
nowatlength = 1
verz = []

# we start here!
b = a.shift  # this gets most left value of array in b, leave the rest in a
b.each { |e|
  verz << [e]
}
# Now verz has 1, 2 of 3 buttons where the first note of the tune is.

puts "Initiated! Now "+a.length.to_s+" notes left to do."
starttime = Time.now

#~ now loop the array by shifting it until zero length
while a.any? do
  
  
  #test
  divv = $divv
  #/test
  
  b = a.shift  
  
  puts "Now looking for "+b.join("-")+"\n"
  
  verz2 = verz.dup

  verz2.each_index { |v|    
    b.cycle(1) { |be|   
        
        #~ do not add notes on a high distance (uppervalscore)
        if $distmat[verz2[v].last][be] < uppervalscore
          verz << [verz2[v],be].flatten(1)  
        else 
          verz.delete(verz2[v])
        end
      }
  } #verz2
    
  #~  now remove the shorter arrays
  #~  todo: adding might be faster than deleting <- it's not. tested 2-12-2018
  nowatlength = nowatlength.next
  verz.delete_if { |v| v.flatten(1).length<nowatlength }
  
  puts "Continue with "+verz.length.to_s+" routes at "+a.length.to_s+" notes to go."
  
  if (verz.length==0)
    raise("No routes found. Raise uppervalscore to higher value (default=30, suggested 120).")
  end

  #  
  # cleanup the array by deleting the worst routes
  #
  if $cleanup && a.length>0
    
    verbose = true #### see on the screen what's going on #############
  
    while (verz.length > $Maxroutes)     
      # lookup the scores, delete all scores that are below '20%' value
      
      prevlength= verz.length.to_s
      
      #calculate scores
      tempscore=Array.new()
      scores=[]

      verz.each_index { |ec|
        e=verz[ec]
        
        sum=calcscore(e)
        
        tempscore[ec] = sum
        scores << sum
        
      } #verz
      

      scores.sort!      
      # puts("Got ",scores.length, " scores.") if verbose 
      
      val20 = scores[((scores.length)/divv).round.to_i]  # these are the sorted scores
      min = scores.min
      max = scores.max
      puts " - Score range: "+min.to_s+"-"+max.to_s+", "+(100/divv).to_i.to_s+"% at "+val20.to_s if verbose 
      
      nwverz = []
      verz.each_index { |ec|
        if calcscore(verz[ec]) <= val20
          e = verz[ec]
          nwverz << e
        end
        }
      
      
      verz = nwverz.dup      ##########TODO
      
      #~ delete if score < 20% value
      #~ 
      puts " - Array shrunk from "+prevlength+" to "+verz.length.to_s+" values." if verbose
      
      if prevlength==verz.length.to_s
        puts(" - No results, adjusting parameter to "+divv.to_s)
        divv = divv + 0.10
      end #if
      
    end #(verz.length)  
  end  #cleanup
end  # while

puts starttime
puts Time.now


Initiated! Now 81 notes left to do.
Now looking for 10-13-19

Continue with 9 routes at 80 notes to go.
Now looking for 27

Continue with 9 routes at 79 notes to go.
Now looking for 4-25-28

Continue with 27 routes at 78 notes to go.
Now looking for 27

Continue with 27 routes at 77 notes to go.
Now looking for 4-25-28

Continue with 81 routes at 76 notes to go.
Now looking for 14-51

Continue with 162 routes at 75 notes to go.
Now looking for 31-32-46

Continue with 486 routes at 74 notes to go.
Now looking for 14-51

Continue with 972 routes at 73 notes to go.
Now looking for 4-25-28

Continue with 2916 routes at 72 notes to go.
Now looking for 10-13-19

Continue with 8748 routes at 71 notes to go.
 - Score range: -73-256, 50% at 93
 - Array shrunk from 8748 to 4436 values.
 - Score range: -73-93, 50% at 9
 - Array shrunk from 4436 to 2220 values.
Now looking for 27

Continue with 2220 routes at 70 notes to go.
Now looking for 4-25-28

Continue with 6660 routes at 69 notes to go.
 - 

In [204]:
$routes=Array.new()

#init 
min=65535
scores=[]

verz.each_index { |ec|
  e=verz[ec]
  sum=calcscore(e)
  
  $routes[ec] = sum
  scores << sum  
} #verz

min  = scores.min
max  = 0
nmax = 999

for i in min...(min+500) do
  puts "Score "+i.to_s+": "+scores.count(i).to_s  if scores.count(i)>0
  max  = max + scores.count(i)
  nmax = i if max <= 20    # if more than 20 routes, stop adding to bestroutes
end



Score -733: 2
Score -731: 4
Score -730: 6
Score -729: 2
Score -728: 42
Score -727: 6
Score -726: 87
Score -725: 108
Score -724: 74
Score -723: 471
Score -722: 129
Score -721: 955
Score -720: 991
Score -716: 4
Score -714: 4
Score -713: 8
Score -712: 2
Score -711: 50
Score -710: 12
Score -709: 93
Score -708: 156
Score -707: 82
Score -706: 604
Score -705: 243
Score -704: 961
Score -703: 997
Score -702: 6
Score -701: 48
Score -700: 8
Score -699: 133
Score -698: 114
Score -694: 2


-733...-233

# best routes

In [205]:
$bestroutes=[] 

verz.each_index{ |ec|
  e = verz[ec]
  if $routes[ec] <= nmax 
    $bestroutes << e
  end  
}

puts "Ready! I have found "+$bestroutes.length.to_s+" paths that have score <= "+nmax.to_s



Ready! I have found 14 paths that have score <= -729


##  Variable   min   contains the lowest score, which is the best path 
##  nmax   is the value with about the best ~20 paths.
##  $bestroutes     is an array containing the best paths. 

In [206]:
# now who is the winner?
winner=[]

$bestroutes.each { |oneroute|
  winner << oneroute if calcscore(oneroute)==min
  }


if false
  sscores=[]
  $bestroutes.each { |oneroute|
    sscores <<  calcscore(oneroute)
    }

  winner=[]
  mmin=scores.min
  $bestroutes.each { |oneroute|
    winner << oneroute if calcscore(oneroute)==mmin
    }
end


# this is the main result of the script

In [207]:
# now we want to now a little more about what we found.
# so, from the best routes
# make a 'median' route


tunelength = $bestroutes[0].length 
nrofbest = $bestroutes.length

puts "these " + nrofbest.to_s + " are the best routes in :"
for i in 0..tunelength-1
  bestnotes = []
  $bestroutes.each_index { |route|
    bestnotes << $bestroutes[route][i]
  }
  
  min=bestnotes.min
  max=bestnotes.max
  
  # Pretty sure about a button when  min==max
  # else there is no obvious winner 
  print "OK:" if min==max
  print "??:" if min!=max
  
  puts bestnotes.group_by { |e| e }.map{ |k, v| [k, v.length] }.to_h.to_s + 
       bestnotes.group_by { |e| e }.map{ |k, v| [k, $tinamap[k]] }.to_h.to_s
    
end 


these 14 are the best routes in :
OK:{25=>14}{25=>"LC1-PULL=A2"}
OK:{10=>14}{10=>"LC1-PUSH=G2"}
OK:{27=>14}{27=>"LG4-PULL=F#2"}
OK:{25=>14}{25=>"LC1-PULL=A2"}
OK:{27=>14}{27=>"LG4-PULL=F#2"}
OK:{25=>14}{25=>"LC1-PULL=A2"}
OK:{51=>14}{51=>"RC1-PULL=B2"}
OK:{32=>14}{32=>"RT2-PUSH=C#3"}
OK:{51=>14}{51=>"RC1-PULL=B2"}
OK:{25=>14}{25=>"LC1-PULL=A2"}
OK:{10=>14}{10=>"LC1-PUSH=G2"}
OK:{27=>14}{27=>"LG4-PULL=F#2"}
??:{25=>2, 28=>12}{25=>"LC1-PULL=A2", 28=>"LG3-PULL=A2"}
??:{52=>2, 15=>12}{52=>"RC2-PULL=D3", 15=>"LG1-PUSH=D3"}
??:{37=>2, 30=>12}{37=>"RC2-PUSH=E3", 30=>"LG1-PULL=E3"}
OK:{56=>14}{56=>"RG1-PULL=F#3"}
OK:{30=>14}{30=>"LG1-PULL=E3"}
OK:{56=>14}{56=>"RG1-PULL=F#3"}
OK:{41=>14}{41=>"RG1-PUSH=G3"}
OK:{56=>14}{56=>"RG1-PULL=F#3"}
OK:{41=>14}{41=>"RG1-PUSH=G3"}
OK:{56=>14}{56=>"RG1-PULL=F#3"}
OK:{30=>14}{30=>"LG1-PULL=E3"}
OK:{56=>14}{56=>"RG1-PULL=F#3"}
OK:{30=>14}{30=>"LG1-PULL=E3"}
OK:{15=>14}{15=>"LG1-PUSH=D3"}
??:{51=>12, 14=>2}{51=>"RC1-PULL=B2", 14=>"LG2-PUSH=B2"}
OK:{25=>14}{25=>

0..81

In [208]:
#
# more details on the
# winner(s) route(s)
# 

puts Time.now

if winner.length<20

  winner.each { |e|
    sum = calcscore(e)
    puts "route:" + e.join(",") + ", score: " + sum.to_s

    e.each_index { |ee|
      fro = e[ee-1]
      too = e[ee]
      line= too.to_s.rjust(2,"0") + "; " + $tinamap[too].to_s + ";"  

      line.include?("#") ? line=line+" " : line=line+"  " # add aspace if no  sharp#
      line = line + fro.to_s.rjust(2,"0") + "->" + too.to_s.rjust(2,"0") + "; " + $distmat[fro][too].to_s.rjust(4," ") + ";" if ee>0
      line = line + " + "   if $distmat[fro][too]>0    # add a '+' 
      puts line
      }
    }
else
  puts "Too much winners ("+ winner.length.to_s+") to list them all!"
end


2018-12-31 15:20:54 +0100
route:25,10,27,25,27,25,51,32,51,25,10,27,28,15,30,56,30,56,41,56,41,56,30,56,30,15,51,25,51,25,27,12,9,25,10,25,51,25,37,56,30,15,30,56,30,56,30,56,30,56,41,57,25,51,25,51,25,27,25,51,25,51,15,30,56,30,56,30,56,30,15,56,57,42,57,56,57,56,30,15,30,15, score: -733
25; LC1-PULL=A2;   + 
10; LC1-PUSH=G2;  25->10;  -19;
27; LG4-PULL=F#2; 10->27;    0;
25; LC1-PULL=A2;  27->25;   -8;
27; LG4-PULL=F#2; 25->27;   -1;
25; LC1-PULL=A2;  27->25;   -8;
51; RC1-PULL=B2;  25->51;   -7;
32; RT2-PUSH=C#3; 51->32;   13; + 
51; RC1-PULL=B2;  32->51;   -7;
25; LC1-PULL=A2;  51->25;   -7;
10; LC1-PUSH=G2;  25->10;  -19;
27; LG4-PULL=F#2; 10->27;    0;
28; LG3-PULL=A2;  27->28;   -1;
15; LG1-PUSH=D3;  28->15;   -7;
30; LG1-PULL=E3;  15->30;  -19;
56; RG1-PULL=F#3; 30->56;   -7;
30; LG1-PULL=E3;  56->30;   -7;
56; RG1-PULL=F#3; 30->56;   -7;
41; RG1-PUSH=G3;  56->41;  -19;
56; RG1-PULL=F#3; 41->56;  -19;
41; RG1-PUSH=G3;  56->41;  -19;
56; RG1-PULL=F#3; 41->56;  -19;
30; LG1-PULL=

[[25, 10, 27, 25, 27, 25, 51, 32, 51, 25, 10, 27, 28, 15, 30, 56, 30, 56, 41, 56, 41, 56, 30, 56, 30, 15, 51, 25, 51, 25, 27, 12, 9, 25, 10, 25, 51, 25, 37, 56, 30, 15, 30, 56, 30, 56, 30, 56, 30, 56, 41, 57, 25, 51, 25, 51, 25, 27, 25, 51, 25, 51, 15, 30, 56, 30, 56, 30, 56, 30, 15, 56, 57, 42, 57, 56, 57, 56, 30, 15, 30, 15], [25, 10, 27, 25, 27, 25, 51, 32, 51, 25, 10, 27, 28, 15, 30, 56, 30, 56, 41, 56, 41, 56, 30, 56, 30, 15, 51, 25, 51, 25, 27, 12, 9, 25, 10, 25, 51, 28, 30, 56, 30, 15, 30, 56, 30, 56, 30, 56, 30, 56, 41, 57, 25, 51, 25, 51, 25, 27, 25, 51, 25, 51, 15, 30, 56, 30, 56, 30, 56, 30, 15, 56, 57, 42, 57, 56, 57, 56, 30, 15, 30, 15]]

In [209]:
# end of script
# concertina-6.rb

In [210]:
# start of plot script

In [211]:
require 'chunky_png'

#globals

#A4 200dpi
$xpapersize = 1654
$ypapersize = 2339  
$tunelength= winner.length

# now xmax depends on papersize and length of tune
# xmax and ymax are the size of one button-system on the page.
#
# let's put  6 systems on $xpapersize and
# let's put 10 systems on $ypapersize and
$nr_butt_x = 6
$nr_butt_y = 10

# and take 5% whitespace, left and right both 2,5%
$xmax = (0.95*$xpapersize/$nr_butt_x).to_i
$ymax = (0.95*$ypapersize/$nr_butt_y).to_i

222

In [212]:
def getxbase(notenumber)
  #
  # xbase is the place of the middle of the diagram on the A4 output
  #
  #
  # width is $xpapersize
  # $nr_butt_x = 6
  # $nr_butt_y = 10

  # $xmax = (0.95*$xpapersize/$nr_butt_x).to_i
  # $ymax = (0.95*$ypapersize/$nr_butt_y).to_i

  # xstart = 0.05 * $xpapersize
  # add $xmax for every system (1..$nr_butt_x)
  # start again ($nr_butt_x+1..$nr_butt_x*2)  

  # midpoint is 
  #  a small whitespace,
  #  the width of one system, when $nr_butt_x systems of x-width
  #  and the middle of the system (half width, xmax/2)

  result = (0.025*$xpapersize + $xmax*(notenumber % $nr_butt_x) + $xmax/2).to_i
end

:getxbase

In [213]:
def getybase(notenumber)
  #
  # ybase is the place of the middle of the diagram on the A4 output
  #
  header = 0 
  result = header + (0.05*$ypapersize + ($ymax*(notenumber/$nr_butt_x))-1).to_i
end

:getybase

In [214]:
def get_x(buttonnr)
  side   = 0
  button = 0
  
  if $tinakeys[buttonnr]["side"] == 2 #right
    side = 1
  else
    side =-1
  end
  button = $tinakeys[buttonnr]["button"]
  
  #             
  # left and right side are both  ($xmax/2) wide
  # button = 1..5   leftside: right to left,   rightside: left to right 
  # side = -1 when leftside,  +1 on rightside
  # a side has 5 buttons, 
  #   take 5,5 of 7 buttons to create some whitespace
  x = side*button*(($xmax/2)/7).to_i
  result = x
end

:get_x

In [215]:
def get_y(buttonnr)   
  #
  # get_y is the positon from the middle of the diagram
  # which is calculated in  getxbase and getybase
  # 
  # xbase and ybase depends n-th note of the tune we are plotting
  # this x and y depend on the button we play
  #
  #
  
  row    = $tinakeys[buttonnr]["row"]      # range 1..3 
  button = $tinakeys[buttonnr]["button"]   # range 1..5
  
  y = 0.05*$ymax +            # whitespace from basepoint
    (row-1)*(0.67*$ymax/3) +  # 3 rows on 67% of the available height
    button*2                  # bends the row a little down
  result = y.to_i
end

:get_y

In [216]:
# Creating an image from scratch, save as an interlaced PNG
# A4 200dpi is 1654 x 2339


# winner = winner[0] if winner.length==1
len = winner.length.to_f
puts winner

ononepage = $nr_butt_x * $nr_butt_y 
nr_of_pages = (len/ononepage).ceil

#  "I have "+ len.to_s+ " notes."
#  "I can handle "+ ononepage.to_s+ " notes on one page."
#  "I need "+ nr_of_pages.to_s+ " pages."

for page in (0..nr_of_pages-1)

  # this page
  startnote = 0             + page*ononepage
  endnote   = (ononepage-1) + page*ononepage
  a_part = winner[startnote..endnote]
  
  
  # create png
  #
  puts " > Created new PNG for page "+page.to_s
  png = ChunkyPNG::Image.new($xpapersize, $ypapersize, ChunkyPNG::Color::WHITE) 
  #
  #
  # now get all the notes of the tune, one by one
  a_part.each_index { |playmenr|   

    
    # the position of the system on the A4-paper
    xbase = getxbase(playmenr)  
    ybase = getybase(playmenr)

    # paint that basepoint # tmp #
    smallb=2
    (xbase-smallb..xbase+smallb).each { |xx|
      (ybase-smallb..ybase+smallb).each { |yy|
        r_colorpp = 0
        g_colorpp = 192
        b_colorpp = 0
        a_colorpp = 255

        # while testing
        # xx= [xx, $xpapersize-1].min
        # yy= [yy, $ypapersize-1].min
        # 
        png[xx,yy] = ChunkyPNG::Color.rgba(r_colorpp, g_colorpp, b_colorpp, a_colorpp)
        }
      }

    
    # the tune at pos playmenr need button... 
    button = a_part[playmenr]  
    
    #
    # loop all buttons, which is too much (duplicate 16..30, 46..60) TODO
    #
    (1..60).each { |nn|

      thisone = false
      howbig  = 6
      
      if button == nn  # paint a bigger button 
        howbig  = 13
        thisone = true  # ok    
      end

      x = xbase + get_x(nn) 
      y = ybase + get_y(nn)

      ######## testing
      verbose = false 
      if verbose
        puts page.to_s+":"+playmenr.to_s+',Button,'+nn.to_s+","+ xbase.to_s+ ","+ybase.to_s+ ","+ x.to_s+ ","+y.to_s+","+ thisone.to_s
      end
      ######## testing

      # get the color fo 
      # not the One:
      r_colorpp = 64
      g_colorpp = 96
      b_colorpp = 128
      a_colorpp = 128

      if thisone         
        if $tinakeys[nn]["ppname"]=="pull"
          r_colorpp = 255  # pull = red
          g_colorpp = 0
          b_colorpp = 0
          a_colorpp = 255  
        else 
          r_colorpp = 0    # push = blue
          g_colorpp = 0
          b_colorpp = 255
          a_colorpp = 255
        end
      end #thisone

      (x-howbig..x+howbig).each { |xx|
        (y-howbig..y+howbig).each { |yy|

          # while testing 
          # xx= [xx, $xpapersize-1].min
          # yy= [yy, $ypapersize-1].min
          #

          png[xx,yy] = ChunkyPNG::Color.rgba(r_colorpp, g_colorpp, b_colorpp, a_colorpp)
          
          
        }
        
      }
      
    }

    #why not add the other button?
    #1. ge tother positions of note f.e A2
    #2. paint a smaller red/blue dot there
  }

  #ready! 
  tunename = tune[:title].to_s
  map = "c:\\write\\"+tunename+"\\"
  filename = map+tunename+"_p"+page.to_s+'.png'
  puts " > Write "+filename
  png.save(filename)

end #page

[[25, 10, 27, 25, 27, 25, 51, 32, 51, 25, 10, 27, 28, 15, 30, 56, 30, 56, 41, 56, 41, 56, 30, 56, 30, 15, 51, 25, 51, 25, 27, 12, 9, 25, 10, 25, 51, 25, 37, 56, 30, 15, 30, 56, 30, 56, 30, 56, 30, 56, 41, 57, 25, 51, 25, 51, 25, 27, 25, 51, 25, 51, 15, 30, 56, 30, 56, 30, 56, 30, 15, 56, 57, 42, 57, 56, 57, 56, 30, 15, 30, 15], [25, 10, 27, 25, 27, 25, 51, 32, 51, 25, 10, 27, 28, 15, 30, 56, 30, 56, 41, 56, 41, 56, 30, 56, 30, 15, 51, 25, 51, 25, 27, 12, 9, 25, 10, 25, 51, 28, 30, 56, 30, 15, 30, 56, 30, 56, 30, 56, 30, 56, 41, 57, 25, 51, 25, 51, 25, 27, 25, 51, 25, 51, 15, 30, 56, 30, 56, 30, 56, 30, 15, 56, 57, 42, 57, 56, 57, 56, 30, 15, 30, 15]]
 > Created new PNG for page 0


TypeError: no implicit conversion of Symbol into Integer

In [217]:
winner

[[25, 10, 27, 25, 27, 25, 51, 32, 51, 25, 10, 27, 28, 15, 30, 56, 30, 56, 41, 56, 41, 56, 30, 56, 30, 15, 51, 25, 51, 25, 27, 12, 9, 25, 10, 25, 51, 25, 37, 56, 30, 15, 30, 56, 30, 56, 30, 56, 30, 56, 41, 57, 25, 51, 25, 51, 25, 27, 25, 51, 25, 51, 15, 30, 56, 30, 56, 30, 56, 30, 15, 56, 57, 42, 57, 56, 57, 56, 30, 15, 30, 15], [25, 10, 27, 25, 27, 25, 51, 32, 51, 25, 10, 27, 28, 15, 30, 56, 30, 56, 41, 56, 41, 56, 30, 56, 30, 15, 51, 25, 51, 25, 27, 12, 9, 25, 10, 25, 51, 28, 30, 56, 30, 15, 30, 56, 30, 56, 30, 56, 30, 56, 41, 57, 25, 51, 25, 51, 25, 27, 25, 51, 25, 51, 15, 30, 56, 30, 56, 30, 56, 30, 15, 56, 57, 42, 57, 56, 57, 56, 30, 15, 30, 15]]